###**Retrieving Sentinel-2 and SRTM data from the Google Earth Engine**

Mapping landslides in Hokkaido, Japan, as a result from an intense earthquake with a magnitude of mw 6.7 occurring on 6 September 2018, in the Iburi-Tobu area of Hokkaido, Japan. This resulted in approximately 6000 landslides occurring with a total area of 46 $km^2$.

This practical script is transformed from a GEE script developed by Muhammad Aufaristama (m.aufaristama@utwente.nl).

In [ ]:
#!pip geopandas

ERROR: unknown command "geopandas"


In [ ]:
#Importing GEE data and folium for opening interactive GEE map
import ee
import folium
#import geopandas 

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [ ]:
#Define datasets that used for landslides mapping
S2 = ee.ImageCollection("COPERNICUS/S2")
DEM = ee.Image("USGS/SRTMGL1_003")
JRC = ee.Image("JRC/GSW1_3/GlobalSurfaceWater")

#Manual mapping inventory produced by Zhang et al 2018 using very high resolution image
table = ee.FeatureCollection("users/aufaristama/landslide_inventory_iburi")

#Define your area, in this case we use Iburi, Hokkaido, Japan
ROI =  ee.Geometry.Polygon(
        [[[141.78777264285623, 42.921828145704644],
          [141.78777264285623, 42.5723893718531],
          [142.1825938098484, 42.5723893718531],
          [142.1825938098484, 42.921828145704644]]], None, False)

#Calculate slope from DEM
terrain = ee.Terrain.products(DEM.select('elevation'))
slope = terrain.select('slope').clip(ROI)        

#Create function to mask water from the image
def maskwater(image):
  water_mask = JRC.select(['max_extent']).lt(1)
  return image.updateMask(water_mask)

#Create function to mask cloud from the image
def maskS2clouds(image):
  qa = image.select('QA60')
  #Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10 
  cirrusBitMask = 1 << 11 
  #Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))
  return image.updateMask(mask) \
      .select("B.*") \
      .copyProperties(image, ["system:time_start"])

#Create function to calculate the NDVI (Normalized Difference Vegetation Index)
def addindices(image):
  ndvi = image.normalizedDifference(['B8', 'B4']).rename('ndvi')
  return image.addBands(ndvi)

#Filter the pre-event image (Before the event)
pre_event = S2 \
  .filter(ee.Filter.date('2015-09-01', '2018-09-05')) \
  .filter(ee.Filter.bounds(ROI)) \
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
  .map(maskwater) \
  .map(maskS2clouds) \
  .map(addindices)
print(pre_event.size().getInfo(), 'pre_event')

#Filter the post-event image (After the event)
post_event = S2 \
  .filter(ee.Filter.date('2018-09-07', '2020-09-01')) \
  .filter(ee.Filter.bounds(ROI)) \
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
  .map(maskwater) \
  .map(maskS2clouds) \
  .map(addindices) 
#print(post_event, 'total_image_post')
print(pre_event.size().getInfo(), 'post_event')

#Calculate relative difference for change detection
#more details on https://nhess.copernicus.org/articles/21/1495/2021/nhess-21-1495-2021.html
rd_subtract = post_event.max().subtract(pre_event.max())
rd_add = pre_event.max().add(post_event.max())
rd_sqrt = rd_add.sqrt()
rd = (rd_subtract.divide(rd_sqrt)).multiply(100)


##Quantify the threshold value, you can delete /* and */ if you already define the sample area and give the name "sample"
sample = ee.Geometry.Polygon(
        [[[141.9017322780952, 42.80632392202265],
          [141.9017322780952, 42.786548149918346],
          [141.92473490260693, 42.786548149918346],
          [141.92473490260693, 42.80632392202265]]], None, False)

#mean
mean = rd.select('ndvi').reduceRegion(reducer= ee.Reducer.mean(),
  geometry= sample,
  scale= 10)
print(mean.getInfo(),'mean')

#standard deviation
stdev = rd.select('ndvi').reduceRegion(reducer= ee.Reducer.stdDev(), 
  geometry= sample,
  scale= 10)
print(stdev.getInfo(),'stdev')

#Threshold min max
thmin = ee.Number(mean.get('ndvi')).subtract(ee.Number(stdev.get('ndvi')))
thmax = ee.Number(mean.get('ndvi')).add(ee.Number(stdev.get('ndvi')))
print(thmin.getInfo(),'thmin')
print(thmax.getInfo(),'thmax')


#Setup the rule/knowledge based thresholds to create binary raster of landslides
landslide_binary = rd.select('ndvi').gt(-64).And(rd.select('ndvi').lt(-25)).And(slope.gt(10)).selfMask()

#Convert the binary into vector(shape_files)
landslide_inventories = landslide_binary.reduceToVectors(
  geometry= ROI,
  scale= 10,
  geometryType= 'polygon',
  eightConnected= False,
  labelProperty= 'zone',
  maxPixels= 1e8)

##Display the results in Google Earth Engine
#Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = "Map Data © Google Earth Engine",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

#Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

#Set visualization parameters for slope.
vis_params = {
  'min': 0,
  'max': 100000000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']
  }

#Set visualization parameters for visible images.
vis_params1 = {
  'min': 0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2']
  }

#Set visualization parameters for NDVI.
vis_params2 = {
  'min': -1,
  'max': 1, #0.85
  'palette': [   'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
  '74A901', '66A000', '529400', '3E8601', '207401', '056201',
  '004C00', '023B01', '012E01', '011D01', '011301']
  }

#Set visualization parameters for landslide binary.
vis_params3 = {
  'min': 0,
  'max': 1,
  'palette': 'red'
  }

#Set visualization parameters for reference data.
vis_params4 = {
  'min': 0,
  'max': 1,
  'palette': ['662A00', 'D8D8D8', 'F5F5F5']
  }

#Create a folium map object with zoom in to location.
location = (42.765669, 142.007348)
my_map = folium.Map(location=location, zoom_start=11)

#Add the slope to the map object.
my_map.add_ee_layer(slope.updateMask(slope.gt(0)), vis_params, 'Slope')

# Add the pre and post event images to the map.
my_map.add_ee_layer(pre_event.median().clip(ROI),vis_params1, 'Visible_pre event')
my_map.add_ee_layer(post_event.median().clip(ROI),vis_params1, 'Visible_post event')
#my_map.add_ee_layer(pre_event.median().clip(ROI).select('B.+'),vis_params1, 'Visible_B_bands_Pre_event')

#Add the pre and post event NDVI's and the relative difference to the map.
my_map.add_ee_layer(pre_event.max().select('ndvi').clip(ROI), vis_params2, 'NDVI_pre event')
my_map.add_ee_layer(post_event.max().select('ndvi').clip(ROI), vis_params2, 'NDVI_post event');
my_map.add_ee_layer(rd.select('ndvi').clip(ROI),vis_params2,'Relative difference (RD) NDVI')

#Adding the landslide_binary layer to the map.
my_map.add_ee_layer(landslide_binary,vis_params3,"Landslide_binary")

#Add the manual mapping table to the map.
#my_map.add_ee_layer(table,vis_params4, 'inventories_Zhang_et_al')
#my_map.add_ee_Layer(landslide_inventories, vis_params4, 'inventories_shapefiles')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

EEException: ignored

## Exporting the pre- and post images to a harddrive.

Exporting function explained on: https://developers.google.com/earth-engine/guides/exporting 

In [ ]:
#Exporting the landslide results to your own Google drive.
ee.batch.Export.table.toDrive(
  collection= landslide_inventories,
  description='inventories_iburi_2018',
  fileFormat= 'SHP')

<Task EXPORT_FEATURES: inventories_iburi_2018 (UNSUBMITTED)>

In [ ]:
#Exporting the pre_event image
task = ee.batch.Export.image.toDrive(image=pre_event.median().select('B.+').visualize(**vis_params1), # an ee.Image object.
                                     region=ROI,  # an ee.Geometry object.
                                     description='pre_event-image_1',
                                     fileNamePrefix='pre_event-image_1',
                                     scale= 10,
                                     maxPixels = 1e10,
                                     fileFormat='GeoTIFF')

#Exporting the pre_event ndvi
task = ee.batch.Export.image.toDrive(image=pre_event.max().select('ndvi'),  # an ee.Image object.
                                     region=ROI,  # an ee.Geometry object.
                                     description='pre_event-ndvi_1',
                                     fileNamePrefix='pre_event-ndvi_1',
                                     scale = 10,
                                     maxPixels = 1e10,
                                     fileFormat='GeoTIFF')

#Exporting the post_event image
task = ee.batch.Export.image.toDrive(image=post_event.median().select('B.+').visualize(**vis_params1), # an ee.Image object.
                                     region=ROI,  # an ee.Geometry object.
                                     description='post_event-image',
                                     fileNamePrefix='post_event-image',
                                     scale= 10,
                                     maxPixels= 1e10,
                                     fileFormat='GeoTIFF')

#Exporting the post_event ndvi
task = ee.batch.Export.image.toDrive(image=post_event.max().select('ndvi'),  # an ee.Image object.
                                     region=ROI,  # an ee.Geometry object.
                                     description='post_event-ndvi_1',
                                     fileNamePrefix='post_event-ndvi_1',
                                     scale = 10,
                                     maxPixels = 1e10,
                                     fileFormat='GeoTIFF')

#Exporting the relative difference (ndvi)
task = ee.batch.Export.image.toDrive(image=rd.select('ndvi'),
                                     region=ROI,
                                     description='Relative_difference',
                                     fileNamePrefix='Relative_difference',
                                     scale = 10,
                                     maxPixels= 1e10,
                                     fileFormat= 'GeoTIFF')
#Exporting the SRTM DEM
task = ee.batch.Export.image.toDrive(image = DEM.select('elevation'),
                                     region = ROI,
                                     description = 'DEM_Hokkaido',
                                     fileNamePrefix = 'DEM_Hokkaido',
                                     scale = 10,
                                     maxPixels = 1e10,
                                     fileFormat = 'GeoTIFF')

#Exporting the Slope
task = ee.batch.Export.image.toDrive(image = slope,
                                     region = ROI,
                                     description = 'Slope_Hokkaido',
                                     fileNamePrefix = 'Slope_Hokkaido',
                                     scale = 10,
                                     maxPixels = 1e10,
                                     fileFormat = 'GeoTIFF')


In [ ]:
task.start()
task.status()

{'creation_timestamp_ms': 1646320841951,
 'description': 'Relative_difference',
 'id': 'H2QPH6V2KNP5EZLYSAQIFMZT',
 'name': 'projects/earthengine-legacy/operations/H2QPH6V2KNP5EZLYSAQIFMZT',
 'start_timestamp_ms': 0,
 'state': 'READY',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1646320841951}